In [1]:
import sys
import os
import toml

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np
import scipy.optimize as SciOpt
from scipy.special import rel_entr
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tqdm.notebook import tqdm
from copy import deepcopy

sys.path.append(os.path.join(sys.path[0], '../..'))

from data.io import Reader
from data.data_generators import DataGenerator
from lsm.lsmtype import Policy
import lsm.cost as CostFunc
import lsm.solver as Solvers

In [2]:
config = Reader.read_config('../../endure.toml')
data_gen = DataGenerator(config)
solver = Solvers.EndureLevelSolver(config)

In [3]:
ewls = config['workloads']
rhos = np.arange(0, 4, 0.5)
ewls

[{'id': 0, 'z0': 0.25, 'z1': 0.25, 'q': 0.25, 'w': 0.25},
 {'id': 1, 'z0': 0.97, 'z1': 0.01, 'q': 0.01, 'w': 0.01},
 {'id': 2, 'z0': 0.01, 'z1': 0.97, 'q': 0.01, 'w': 0.01},
 {'id': 3, 'z0': 0.01, 'z1': 0.01, 'q': 0.97, 'w': 0.01},
 {'id': 4, 'z0': 0.01, 'z1': 0.01, 'q': 0.01, 'w': 0.97},
 {'id': 5, 'z0': 0.49, 'z1': 0.49, 'q': 0.01, 'w': 0.01},
 {'id': 6, 'z0': 0.49, 'z1': 0.01, 'q': 0.49, 'w': 0.01},
 {'id': 7, 'z0': 0.49, 'z1': 0.01, 'q': 0.01, 'w': 0.49},
 {'id': 8, 'z0': 0.01, 'z1': 0.49, 'q': 0.49, 'w': 0.01},
 {'id': 9, 'z0': 0.01, 'z1': 0.49, 'q': 0.01, 'w': 0.49},
 {'id': 10, 'z0': 0.01, 'z1': 0.01, 'q': 0.49, 'w': 0.49},
 {'id': 11, 'z0': 0.33, 'z1': 0.33, 'q': 0.33, 'w': 0.01},
 {'id': 12, 'z0': 0.33, 'z1': 0.33, 'q': 0.01, 'w': 0.33},
 {'id': 13, 'z0': 0.33, 'z1': 0.01, 'q': 0.33, 'w': 0.33},
 {'id': 14, 'z0': 0.01, 'z1': 0.33, 'q': 0.33, 'w': 0.33}]

In [4]:
def sample_workloads(samples=10000):
    wls = []
    for _ in range(samples):
        wls.append(data_gen._sample_workload(4))
    return wls

In [5]:
def create_tunings():
    df = []
    pbar = tqdm(ewls, desc='Workloads')

    for idx, w in enumerate(pbar):
        pbar.set_postfix(workload=w)
        row = {}
        row['workload_idx'] = idx
        z0, z1, q, w = (w['z0'], w['z1'], w['q'], w['w'])
        row['z0'], row['z1'], row['q'], row['w'] = (z0, z1, q, w)
        for key in config['system'].keys():
            row[key] = config['system'][key]

        nominal = solver.find_nominal_design(z0, z1, q, w)
        row['nominal_h'] = h_init = nominal.x[0]
        row['nominal_T'] = T_init = nominal.x[1]
        # row['nominal_Q'] = Q_init = nominal.x[2]
        row['nominal_cost'] = nominal.fun
        row['nominal_msg'] = nominal.message
        for rho in rhos:
            row['rho'] = rho
            # robust = solver.find_robust_design(rho, z0, z1, q, w, [h_init, T_init, Q_init])
            robust = solver.find_robust_design(rho, z0, z1, q, w, [h_init, T_init])
            row['robust_h'] = robust.x[0]
            row['robust_T'] = robust.x[1]
            # row['robust_Q'] = robust.x[2]
            row['robust_msg'] = robust.message
            # row['robust_cost'] = solver.nominal_objective(robust.x[0:3], z0, z1, q, w)
            row['robust_cost'] = solver.nominal_objective(robust.x[0:2], z0, z1, q, w)

            # Append the design to the dataframe
            df.append(deepcopy(row))

    df = pd.DataFrame(df)
    return df

In [6]:
tunings = create_tunings()
tunings

Workloads:   0%|          | 0/15 [00:00<?, ?it/s]

/Users/ndhuynh/miniforge3/lib/python3.9/site-packages/scipy/optimize/_optimize.py:284: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/ndhuynh/repos/endure/endure-torch/notebook/data-gen/../../lsm/solver.py:23: RuntimeWarning: overflow encountered in exp
  return np.exp(input) - 1


,workload_idx,z0,z1,q,w,B,phi,s,E,H,N,nominal_h,nominal_T,nominal_cost,nominal_msg,rho,robust_h,robust_T,robust_msg,robust_cost
0,0,0.25,0.25,0.25,0.25,4,1,0.00,8192,10,10000000,3.76,6.02,2.90,Optimization terminated successfully,0.00,3.76,6.02,Inequality constraints incompatible,2.90
1,0,0.25,0.25,0.25,0.25,4,1,0.00,8192,10,10000000,3.76,6.02,2.90,Optimization terminated successfully,0.50,3.76,6.02,Inequality constraints incompatible,2.90
2,0,0.25,0.25,0.25,0.25,4,1,0.00,8192,10,10000000,3.76,6.02,2.90,Optimization terminated successfully,1.00,3.76,6.02,Inequality constraints incompatible,2.90
3,0,0.25,0.25,0.25,0.25,4,1,0.00,8192,10,10000000,3.76,6.02,2.90,Optimization terminated successfully,1.50,3.76,6.02,Inequality constraints incompatible,2.90
4,0,0.25,0.25,0.25,0.25,4,1,0.00,8192,10,10000000,3.76,6.02,2.90,Optimization terminated successfully,2.00,3.76,6.02,Inequality constraints incompatible,2.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,14,0.01,0.33,0.33,0.33,4,1,0.00,8192,10,10000000,2.39,4.04,3.78,Optimization terminated successfully,1.50,2.39,4.04,Inequality constraints incompatible,3.78
116,14,0.01,0.33,0.33,0.33,4,1,0.00,8192,10,10000000,2.39,4.04,3.78,Optimization terminated successfully,2.00,2.39,4.04,Inequality constraints incompatible,3.78
117,14,0.01,0.33,0.33,0.33,4,1,0.00,8192,10,10000000,2.39,4.04,3.78,Optimization terminated successfully,2.50,2.39,4.04,Inequality constraints incompatible,3.78
118,14,0.01,0.33,0.33,0.33,4,1,0.00,8192,10,10000000,2.39,4.04,3.78,Optimization terminated successfully,3.00,2.39,4.04,Inequality constraints incompatible,3.78


In [12]:
tunings.nominal_msg.unique()

array(['Optimization terminated successfully'], dtype=object)

In [13]:
tunings.groupby('workload_idx').get_group(11)

,workload_idx,z0,z1,q,w,B,phi,s,E,H,N,nominal_h,nominal_T,nominal_cost,nominal_msg,rho,robust_h,robust_T,robust_msg,robust_cost
88,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,0.00,7.49,7.56,Inequality constraints incompatible,1.82
89,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,0.50,7.49,7.56,Inequality constraints incompatible,1.82
90,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,1.00,7.49,7.56,Inequality constraints incompatible,1.82
91,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,1.50,7.49,7.56,Inequality constraints incompatible,1.82
92,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,2.00,7.49,7.56,Inequality constraints incompatible,1.82
93,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,2.50,7.49,7.56,Inequality constraints incompatible,1.82
94,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,3.00,7.49,7.56,Inequality constraints incompatible,1.82
95,11,0.33,0.33,0.33,0.01,4,1,0.00,8192,10,10000000,7.49,7.56,1.82,Optimization terminated successfully,3.50,7.49,7.56,Inequality constraints incompatible,1.82


In [8]:
df = pd.read_csv('/Users/ndhuynh/sandbox/data/workload_uncertainty_tunings.csv')
df.groupby('workload_idx').get_group(11)

,workload_idx,z0,z1,q,w,N,phi,B,s,E,...,nominal_cost,nominal_is_leveling_policy,rho,robust_exit_mode,robust_m_h,robust_m_filt,robust_m_buff,robust_T,robust_cost,robust_is_leveling_policy
88,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,0.00,0,4.44,"44,420,621.94","55,579,378.06",46.72,1.30,True
89,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,0.50,0,1.35,"13,457,859.45","86,542,140.55",10.33,1.70,True
90,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,1.00,0,1.00,"10,000,000.00","90,000,000.00",8.55,1.83,True
91,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,1.50,0,1.00,"10,000,000.00","90,000,000.00",7.34,1.92,True
92,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,2.00,0,1.00,"10,000,000.00","90,000,000.00",5.58,2.14,True
93,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,2.50,0,1.00,"10,000,000.00","90,000,000.00",5.58,2.14,True
94,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,3.00,0,1.00,"10,000,000.00","90,000,000.00",5.58,2.14,True
95,11,0.33,0.33,0.33,0.01,10000000,1,4,0.00,8192,...,1.30,True,3.50,0,1.00,"10,000,000.00","90,000,000.00",5.58,2.14,True


In [9]:
samples = sample_workloads()
distances = {}
for idx, wl in enumerate(ewls):
    key = idx
    w0 = [wl['z0'], wl['z1'], wl['q'], wl['w']]
    distances[key] = []
    for sample in samples:
        distances[key].append(np.sum(rel_entr(sample, w0)))

In [10]:
# comparisons = []
# for tuning in tqdm(tunings.to_records('dict'), desc='Tunings'):
#     row = {}
#     row['workload_idx'] = tuning['workload_idx']
#     row['z0'] = tuning['z0']
#     row['z1'] = tuning['z1']
#     row['q'] = tuning['q']
#     row['w'] = tuning['w']
#     row['robust_rho'] = tuning['rho']
#     row['robust_h'] = tuning['robust_h']
#     row['robust_T'] = tuning['robust_T']
#     row['robust_Q'] = tuning['robust_Q']

#     row['nominal_h'] = tuning['nominal_h']
#     row['nominal_T'] = tuning['nominal_T']
#     row['nominal_Q'] = tuning['nominal_Q']
#     nominal = [row['nominal_h'], row['nominal_T'], row['nominal_Q']]
#     robust = [row['robust_h'], row['robust_T'], row['robust_Q']]

#     distance = distances[row['workload_idx']]
#     for idx, w_hat in enumerate(samples):
#         row['rho_hat'] = distance[idx]
#         row['z0_hat'] = w_hat[0]
#         row['z1_hat'] = w_hat[1]
#         row['q_hat'] = w_hat[2]
#         row['w_hat'] = w_hat[3]
#         row['sample_idx'] = idx
#         row['nominal_cost'] = solver.nominal_objective(nominal, *w_hat)
#         row['robust_cost'] = solver.nominal_objective(robust, *w_hat)
#         comparisons.append(deepcopy(row))

# df = pd.DataFrame(comparisons)

In [11]:
# df.to_parquet('/Users/ndhuynh/sandbox/data/exp01.parquet', index=False)